EDA

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import yfinance as yf
import numpy as np
from sklearn.preprocessing import StandardScaler

We import the stock price of Starbucks(SBUX) and Apple(AAPL) and the industrial index of S&P 500(SPY).

In [2]:
spy= yf.download(
        tickers = "SPY",
        period = "5d",
        interval = "1m",
        group_by = 'ticker',
        auto_adjust = True,
        prepost = False,
        threads = True,
        proxy = None)
spy.reset_index(inplace=True,drop=False)
spy.rename(columns = {'Close':'SPY'}, inplace = True) 
spy=spy.drop(['Open', 'High','Low','Volume'], axis=1)


sbux= yf.download(
        tickers = "SBUX",
        period = "5d",
        interval = "1m",
        group_by = 'ticker',
        auto_adjust = True,
        prepost = False,
        threads = True,
        proxy = None)
sbux.reset_index(inplace=True,drop=False)
sbux=sbux.drop(['Open', 'High','Low','Volume','Datetime'], axis=1)
sbux.rename(columns = {'Close':'SBUX'}, inplace = True) 

aapl= yf.download(
        tickers = "AAPL",
        period = "5d",
        interval = "1m",
        group_by = 'ticker',
        auto_adjust = True,
        prepost = False,
        threads = True,
        proxy = None)
aapl.reset_index(inplace=True,drop=False)
aapl=aapl.drop(['Open', 'High','Low','Volume','Datetime'], axis=1)
aapl.rename(columns = {'Close':'AAPL'}, inplace = True);

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
data = pd.concat([spy, sbux,aapl], axis=1, sort=False)

In [4]:
data

,Datetime,SPY,SBUX,AAPL
0,2020-11-30 09:30:00-05:00,362.839996,97.900002,117.389999
1,2020-11-30 09:31:00-05:00,362.890015,97.750000,117.589996
2,2020-11-30 09:32:00-05:00,362.890015,97.650002,118.040001
3,2020-11-30 09:33:00-05:00,362.970001,97.639999,118.087700
4,2020-11-30 09:34:00-05:00,362.750000,97.605003,118.239998
...,...,...,...,...
1730,2020-12-04 12:28:00-05:00,369.040009,102.714996,121.876701
1731,2020-12-04 12:29:00-05:00,369.025604,102.730003,121.879997
1732,2020-12-04 12:30:00-05:00,369.068115,102.684998,121.910103
1733,2020-12-04 12:31:00-05:00,369.149902,102.650002,122.029999


ETL

In [18]:
import pymongo
client = pymongo.MongoClient()

In [19]:
client.test_database

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test_database')

In [20]:
db = client.get_database("stock")
collection = db.get_collection("stock")
update_count = 0
for record in data.to_dict('records'):
    result = collection.replace_one(
        filter={'Datetime': record['Datetime']},    
        replacement=record,                         
        upsert=True)                                
    if result.matched_count > 0:
        update_count += 1
print(f"rows={data.shape[0]}, update={update_count}, "
      f"insert={data.shape[0]-update_count}")

rows=1735, update=0, insert=1735
